In [1]:
from __future__ import division, print_function

import os, json
from glob import glob
import numpy as np
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, load_model,  Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
from keras.preprocessing import image
from keras.applications import VGG16
from keras.layers import Input, Lambda, Cropping2D, Activation, ELU
from keras.layers.merge import add, concatenate
from keras.models import model_from_json

%matplotlib inline

Using TensorFlow backend.


In [2]:
#from directkeys import PressKey,ReleaseKey, W, A, S, D
from utils import *

In [17]:
# add time for key to keep pressed
t_time = 0.11

def straight():
    PressKey(W)
    ReleaseKey(A)
    ReleaseKey(D)
    
def left():
    PressKey(W)
    PressKey(A)
    ReleaseKey(D)
    time.sleep(t_time)
    ReleaseKey(A)
    #ReleaseKey(W)

def right():
    PressKey(W)
    PressKey(D)
    ReleaseKey(A)
    time.sleep(t_time)
    ReleaseKey(D)
    #ReleaseKey(W)

In [4]:
results_path = 'D:/ML_DATA/wheelai/gtaV/results/'

In [7]:
def get_model(model, weights):
    model_path = results_path + model
    weights = results_path + weights
    # load modvvel
    json_file = open(model_path, 'r')
    model_json = json_file.read()
    json_file.close()
    model = model_from_json(model_json)
    # load weights into new model
    model.load_weights(weights)
    print("Loaded model from disk")
    # compile model
    adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(optimizer=adam,loss='categorical_crossentropy')
    return model

In [12]:
model = get_model('nvidia_conv.json', 'nvidia_conv6.h5')

Loaded model from disk


In [20]:
def main():
    count_down(5)
    paused = False
    while(True):

        if not paused:
            # get the image
            img = grab_screen(region=(0, 100, 1280, 640))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # resize image depending on your model specs
            img = cv2.resize(img,(320, 160))
            # preprocess the image
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            action = model.predict(x, batch_size=1,verbose=0)[0]
            print(action)
            # get predicted action i.e action with highest probability 
            max_index = np.argmax(action)
                       
            if max_index == 1:
                left()
            elif max_index == 2:
                right()
            else:
                straight()
        # pause - unpause the agent
        keys = key_check()
        if 'T' in keys:
            if paused:
                paused = False
                print('unpaused!')
                time.sleep(1)
            else:
                print('Pausing!')
                paused = True
                ReleaseKey(A)
                ReleaseKey(W)
                ReleaseKey(D)
                time.sleep(1)


In [24]:
main()

5
4
3
2
1
[ 0.02407148  0.92318648  0.05274206]
[ 0.01276914  0.80669343  0.18053751]
[ 0.03383185  0.68964064  0.27652752]
[ 0.03383185  0.68964064  0.27652752]
[ 0.11903878  0.72255355  0.15840772]
[ 0.14981027  0.65642935  0.19376035]
[ 0.20762548  0.66681236  0.12556219]
[ 0.20762548  0.66681236  0.12556219]
[ 0.21171425  0.65866292  0.12962286]
[ 0.15166369  0.45457184  0.39376444]
[ 0.00926146  0.79650217  0.19423637]
[ 0.02188339  0.77070278  0.20741379]
[ 0.05140151  0.20238268  0.74621588]
[ 0.09279636  0.08967181  0.81753182]
[ 0.12202245  0.44233578  0.43564174]
[ 0.03755255  0.91542864  0.04701879]
[ 0.02992528  0.88975608  0.08031862]
[ 0.09526993  0.06574062  0.8389895 ]
[ 0.08528466  0.03715095  0.87756437]
[ 0.03390318  0.01382254  0.95227426]
[ 0.11226956  0.21493414  0.67279631]
[ 0.22870708  0.23615405  0.53513891]
[ 0.11159669  0.40209004  0.48631325]
[ 0.02747919  0.68097723  0.29154354]
[ 0.01551531  0.89169717  0.09278751]
[ 0.02543107  0.87507409  0.09949478]
[ 

[ 0.08564445  0.06302502  0.85133052]
[ 0.13408373  0.31159043  0.55432582]
[ 0.12725979  0.42656446  0.44617578]
[ 0.05634927  0.40586644  0.53778428]
[ 0.12753789  0.5834884   0.28897366]
[ 0.09474117  0.8312667   0.07399212]
[ 0.02832811  0.90523237  0.06643957]
[ 0.0191041   0.85631096  0.12458493]
[ 0.05142394  0.46115804  0.487418  ]
[ 0.08085451  0.3431268   0.57601863]
[ 0.08707438  0.61685157  0.29607406]
[ 0.03730111  0.88389128  0.07880762]
[ 0.08017236  0.45806736  0.46176034]
Pausing!
unpaused!
[ 0.01834727  0.96637529  0.01527754]
[ 0.04494836  0.93238074  0.02267097]
[ 0.06873904  0.87812722  0.05313378]
[ 0.07537501  0.86319959  0.06142547]
[ 0.1334196   0.65278041  0.2138    ]
[ 0.32557371  0.26819038  0.40623593]
[ 0.12977263  0.12255716  0.74767017]
[ 0.12713876  0.13495083  0.73791039]
[ 0.23915285  0.30519241  0.45565477]
[ 0.02866997  0.93259966  0.03873029]
[ 0.00535746  0.9888671   0.00577537]
[ 0.02273479  0.96204054  0.01522465]
[ 0.22979219  0.56052142  0.209

[ 0.06390851  0.82488537  0.11120611]
[ 0.04809764  0.81144375  0.14045855]
[ 0.03854212  0.82840222  0.13305569]
[ 0.03308564  0.78153139  0.18538301]
[ 0.03784866  0.75131506  0.21083626]
[ 0.04034446  0.73915446  0.2205011 ]
[ 0.03520745  0.68111098  0.28368154]
[ 0.03436155  0.55268854  0.41294992]
[ 0.03485676  0.67967248  0.28547072]
[ 0.05233676  0.63768208  0.30998111]
[ 0.05218019  0.58328074  0.36453909]
[ 0.03685178  0.63483918  0.32830909]
[ 0.02060807  0.67126071  0.30813125]
[ 0.04029017  0.76171809  0.19799171]
[ 0.03771326  0.58437204  0.37791473]
[ 0.03957193  0.63206041  0.32836768]
[ 0.04259748  0.68944228  0.26796022]
[ 0.04600851  0.71893555  0.23505594]
[ 0.07131571  0.73530525  0.19337901]
[ 0.05600755  0.72816682  0.21582554]
[ 0.0737311   0.606345    0.31992391]
[ 0.04526731  0.76562136  0.18911126]
[ 0.0760239   0.67821991  0.24575616]
Pausing!
unpaused!
[ 0.0056907   0.99294597  0.00136335]
[ 0.00651343  0.99185973  0.00162688]
[  6.55649928e-03   9.92499888e

KeyboardInterrupt: 